<a href="https://colab.research.google.com/github/asheta66/CNN/blob/main/Chest%20X_Ray/Chest_X_Ray_by_CNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================================
# CNN CHEST X-RAY CLASSIFICATION (PURE TENSORFLOW)
# NORMAL vs PNEUMONIA | Full Evaluation
# ============================================================

import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, BatchNormalization,
    Flatten, Dense, Dropout
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import (
    confusion_matrix, accuracy_score,
    precision_score, recall_score, f1_score,
    roc_curve, auc
)

# ----------------------------
# 1. Configuration
# ----------------------------
IMG_SIZE = (224, 224)
BATCH_SIZE = 128
EPOCHS = 10
LR = 1e-3
DATA_DIR = "/content/drive/MyDrive/Chest X_Ray"
SEED = 22

tf.random.set_seed(SEED)
np.random.seed(SEED)

# ----------------------------
# 2. Data Generators
# ----------------------------
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training",
    shuffle=True,
    seed=SEED
)

val_data = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation",
    shuffle=False
)

# ----------------------------
# 3. CNN Model Definition
# ----------------------------
from tensorflow.keras.layers import Input

model = Sequential([
    Input(shape=(*IMG_SIZE, 3)),

    Conv2D(32, 3, activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(64, 3, activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(128, 3, activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Conv2D(256, 3, activation="relu"),
    BatchNormalization(),
    MaxPooling2D(),

    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")
])

model.compile(
    optimizer=Adam(learning_rate=LR),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# model.summary()

# ----------------------------
# 4. Callbacks
# ----------------------------
callbacks = [
    EarlyStopping(patience=4, restore_best_weights=True),
    ReduceLROnPlateau(patience=2, factor=0.3, verbose=1)
]

# ----------------------------
# 5. Training
# ----------------------------
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

# ----------------------------
# 6. Evaluation Function
# ----------------------------
def evaluate_generator(model, generator):
    generator.reset()
    y_true = generator.classes
    y_prob = model.predict(generator, verbose=0).ravel()
    y_pred = (y_prob > 0.5).astype(int)

    metrics = {
        "Accuracy": accuracy_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred)
    }

    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    roc_auc = auc(fpr, tpr)

    return metrics, cm, fpr, tpr, roc_auc

# ----------------------------
# 7. Train & Test Evaluation
# ----------------------------
train_metrics, cm_train, fpr_train, tpr_train, auc_train = evaluate_generator(model, train_data)
test_metrics,  cm_test,  fpr_test,  tpr_test,  auc_test  = evaluate_generator(model, val_data)


Found 4686 images belonging to 2 classes.
Found 1170 images belonging to 2 classes.
Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 1027s 27s/step - accuracy: 0.8313 - loss: 3.3124 - val_accuracy: 0.7299 - val_loss: 12.3688 - learning_rate: 0.0010
Epoch 2/10
28/37 ━━━━━━━━━━━━━━━━━━━━ 3:41 25s/step - accuracy: 0.9513 - loss: 0.1408

In [ ]:
# ----------------------------
# 8. Metrics Table
# ----------------------------
df_results = pd.DataFrame([
    ["Train", *train_metrics.values()],
    ["Test",  *test_metrics.values()]
], columns=["Dataset", "Accuracy", "Precision", "Recall", "F1"])

print("\nPerformance Summary:")
display(df_results)

# ----------------------------
# 9. Confusion Matrices (Subplots)
# ----------------------------
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.imshow(cm_train, cmap="Blues")
plt.title("Train Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(cm_test, cmap="Blues")
plt.title("Test Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.colorbar()

plt.tight_layout()
plt.show()

# ----------------------------
# 10. ROC Curves (Single Plot)
# ----------------------------
plt.figure(figsize=(6,6))

plt.plot(fpr_train, tpr_train, label=f"Train ROC (AUC = {auc_train:.3f})", linewidth=2)
plt.plot(fpr_test,  tpr_test,  label=f"Test ROC (AUC = {auc_test:.3f})", linewidth=2)
plt.plot([0,1], [0,1], 'k--')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – CNN (TensorFlow)")
plt.legend(loc="lower right")
plt.grid(True)
plt.tight_layout()
plt.show()
